In [2]:
import pandas as pd

# Функции для предобработки данных
def add_id_prefix(subs_id):
    return f'id{subs_id}' if not str(subs_id).startswith('id') else subs_id

def convert_date_format(date_string):
    try:
        return pd.to_datetime(date_string, format='%d-%m-%Y %H:%M')
    except:
        return pd.NaT

# Загрузка файлов продаж и логов активаций
sales_file_1_path = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-zakorko/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/tm_sales_1.csv'
sales_file_2_path = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-zakorko/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/tm_sales_2.csv'
sales_file_3_path = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-zakorko/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/tm_sales_3.csv'
activations_logs_path = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-a-zakorko/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/prod_activations_logs.csv'

sales_data_1 = pd.read_csv(sales_file_1_path, sep=';')
sales_data_2 = pd.read_csv(sales_file_2_path, sep=';')
sales_data_3 = pd.read_csv(sales_file_3_path, sep=';')
activations_logs_data = pd.read_csv(activations_logs_path, sep=';')

# Предобработка данных продаж
sales_data_1['SUBS_ID'] = sales_data_1['SUBS_ID'].apply(add_id_prefix)
sales_data_2['SUBS_ID'] = sales_data_2['SUBS_ID'].apply(add_id_prefix)
sales_data_3['SUBS_ID'] = sales_data_3['SUBS_ID'].apply(add_id_prefix)

sales_data_1.dropna(subset=['SUBS_ID'], inplace=True)
sales_data_2.dropna(subset=['SUBS_ID'], inplace=True)
sales_data_3.dropna(subset=['SUBS_ID'], inplace=True)

sales_data_1['ACT_DTTM'] = sales_data_1['ACT_DTTM'].apply(convert_date_format)
sales_data_2['ACT_DTTM'] = sales_data_2['ACT_DTTM'].apply(convert_date_format)
sales_data_3['ACT_DTTM'] = sales_data_3['ACT_DTTM'].apply(convert_date_format)

# Предобработка данных логов активаций
activations_logs_data['START_DTTM'] = activations_logs_data['START_DTTM'].apply(convert_date_format)
activations_logs_data['END_DTTM'] = activations_logs_data['END_DTTM'].apply(convert_date_format)

# Объединение файлов продаж
combined_sales_data = pd.concat([sales_data_1, sales_data_2, sales_data_3], ignore_index=True)

# Объединение данных продаж с логами активаций
merged_data = pd.merge(combined_sales_data, activations_logs_data, on=['SUBS_ID', 'PROD_ID'], how='inner')

# Расчет разницы между START_DTTM и END_DTTM
merged_data['difference'] = merged_data['END_DTTM'] - merged_data['START_DTTM']

# Фильтрация продаж, где разница меньше 5 минут
filtered_data = merged_data[merged_data['difference'] >= pd.Timedelta(minutes=5)]

# Получение и сортировка SUBS_ID из отфильтрованных данных
sorted_subs_ids = sorted(filtered_data['SUBS_ID'].unique())
sorted_subs_ids_str = ', '.join(sorted_subs_ids)

sorted_subs_ids_str


'id2185490, id4651830, id5416547, id7642700'